# PFN Internship 2022 "JE06. Development of algorithms to calculate physical properties for Matlantis" Report Assignment

Work on the following optimization problem. This problem is written in Python.

If you find it easier to work on the task of "JE19. Applied research and development of machine learning and atomic simulation on materials" easier to work on than this assignment, you may answer that assignment instead of this one.

## Problem Setup

Place 64 labeled points in 3-dimensional space. The positions of the points are assumed to be non-overlapping. The labels are integer values. For each integer from 0 to 7, there are 8 points with a label with a value equal to that integer. Consider a function that returns a scalar value for this set of points. In this case, consider the arrangement of the points such that the return value is as small as possible.

This assignment is modeled after the problem of estimating atomic structure. By considering each point as an atom and the scalar value as energy, the problem can be regarded as a problem of finding an atomic arrangement that is low in energy and stable.

## About Function

The concrete form of the function is provided below as the `calculate` function of the `LennardJonesI22` class. The details are described below, but here you can consider it as a black-box function.

It takes two arguments: atom_type (64-dimensional, integer), which corresponds to the label of the point, and positions (64x3-dimensional, floating-point number), which corresponds to the coordinates of the point. The return values are `output` (a scalar value) and `grad` (the derivative of the output with respect to the coordinates). You don't have to use `grad`.

This is what is known as the Lennard-Jones potential. For all point combinations, it is the sum of the return values of a function defined by the distance between two points and the labels of the two points as input arguments (two-body function).

The two-body function asymptotes to zero when the distances are far enough apart. It behaves in such a way that its value decreases as the distance approaches a certain length and rapidly increases as the distance approaches further. Therefore, a good solution is to place each point at some stable distance from the other. The stable distance is about between 2 and 4.


In [1]:
from typing import Tuple

import numpy as np


class LennardJonesI22:
    def __init__(self):
        sigma_single = np.array([2.0 + 0.2 * x for x in range(8)])
        epsilon_single = np.array([0.1 + 0.1 * x for x in range(8)])

        self.sigma_matrix = 0.5 * (sigma_single[None, :] + sigma_single[:, None])
        self.epsilon_matrix = np.sqrt(epsilon_single[None, :] * epsilon_single[:, None])

    def calculate(self, atom_type: np.ndarray, positions: np.ndarray) -> Tuple[float, np.ndarray]:
        assert len(positions.shape) == 2
        assert positions.shape[1] == 3
        n_atoms = positions.shape[0]
        
        assert len(atom_type.shape) == 1
        assert atom_type.shape[0] == n_atoms

        an_axis0 = np.repeat(atom_type[None, :], n_atoms, axis=0)
        an_axis1 = np.repeat(atom_type[:, None], n_atoms, axis=1)
        sigma_pairs = self.sigma_matrix[an_axis0, an_axis1]
        epsilon_pairs = self.epsilon_matrix[an_axis0, an_axis1]
    
        x1 = positions[None, :, :]
        x2 = positions[:, None, :]
        x_diff = x2 - x1
        rsq = np.sum(np.square(x_diff), axis=2)
        rsq_reciprocal = np.reciprocal(rsq, out=np.zeros_like(rsq), where=(rsq != 0.0))

        sigma_by_r2 = np.square(sigma_pairs) * rsq_reciprocal
        sigma_by_r6 = np.power(sigma_by_r2, 3)
        sigma_by_r12 = np.square(sigma_by_r6)
        e_pairs = 4.0 * epsilon_pairs * (sigma_by_r12 - sigma_by_r6)
        f_pairs_by_r = -24.0 * epsilon_pairs * (2 * sigma_by_r12 - sigma_by_r6) * rsq_reciprocal
        f_pairs = f_pairs_by_r[:, :, None] * x_diff
    
        output_atoms = 0.5 * np.sum(e_pairs, axis=1)
        output = float(np.sum(output_atoms).item())
        if np.any(rsq + np.identity(rsq.shape[0]) == 0.0):  # Same position
            output = float("inf")
        grad = 0.5 * (np.sum(f_pairs, axis=1) - np.sum(f_pairs, axis=0))
        
        return output, grad

This can be used to calculate a scalar value (energy) for an appropriate input. Below is an example calculation for a straight line configuration in 3-dimensional space.

In [2]:
atom_type = np.array([0, 1, 2, 3, 4, 5, 6, 7] * 8)
positions = np.array([[3.5 * x, 0.0, 0.0] for x in range(64)])

f = LennardJonesI22()
f_out, _ = f.calculate(atom_type, positions)
print("output: {:.5f}".format(f_out))

print("atom_type")
print(atom_type.tolist())
print("positions")
print(positions.tolist())

output: -20.32051
atom_type
[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7]
positions
[[0.0, 0.0, 0.0], [3.5, 0.0, 0.0], [7.0, 0.0, 0.0], [10.5, 0.0, 0.0], [14.0, 0.0, 0.0], [17.5, 0.0, 0.0], [21.0, 0.0, 0.0], [24.5, 0.0, 0.0], [28.0, 0.0, 0.0], [31.5, 0.0, 0.0], [35.0, 0.0, 0.0], [38.5, 0.0, 0.0], [42.0, 0.0, 0.0], [45.5, 0.0, 0.0], [49.0, 0.0, 0.0], [52.5, 0.0, 0.0], [56.0, 0.0, 0.0], [59.5, 0.0, 0.0], [63.0, 0.0, 0.0], [66.5, 0.0, 0.0], [70.0, 0.0, 0.0], [73.5, 0.0, 0.0], [77.0, 0.0, 0.0], [80.5, 0.0, 0.0], [84.0, 0.0, 0.0], [87.5, 0.0, 0.0], [91.0, 0.0, 0.0], [94.5, 0.0, 0.0], [98.0, 0.0, 0.0], [101.5, 0.0, 0.0], [105.0, 0.0, 0.0], [108.5, 0.0, 0.0], [112.0, 0.0, 0.0], [115.5, 0.0, 0.0], [119.0, 0.0, 0.0], [122.5, 0.0, 0.0], [126.0, 0.0, 0.0], [129.5, 0.0, 0.0], [133.0, 0.0, 0.0], [136.5, 0.0, 0.0], [140.0, 0.0, 0.0], [143.5, 0.0, 0.0], [1

## What to submit

Please submit a report (no more than one sheet of A4 paper in PDF format, or a little over a sheet if you cannot write on one sheet) describing the results (input pairs and their scores) and how you thought about them, as well as the source code of the program used to run the simulation. You may use the distributed Jupyter Notebook as-is for your program, or you may use a different format.

For this assignment, you do not need to spend a lot of time and work down to the very last minute on your score. Instead, please consider readability when writing your program. Also, be sure to write in your report what you have considered in your optimization.

## Supplementary: How to solve

You are free to solve in any way you like. You may use various gradient methods or optimization functions from neural network libraries such as PyTorch using derivative values, or you may consider it a black box optimization problem and use a black box optimization library such as Optuna.

Those with knowledge of atomic simulation may also use various optimization methods in molecular dynamics calculations, such as MCMC and annealing methods, to solve the problem. Below is an example calculation using the Python atomic simulation library ASE. (You need to install ASE in your environment, e.g. `pip install ase`).

In [3]:
import numpy as np
import ase
import ase.optimize
from ase.calculators.calculator import Calculator, all_changes


class LennardJonesI22Calculator(Calculator):
    implemented_properties = ['energy', 'forces', 'free_energy']
    
    def __init__(self, **kwargs):
        Calculator.__init__(self, **kwargs)
        self.lj_core = LennardJonesI22()
        
    def calculate(self, atoms=None, properties=None, system_changes=all_changes):
        if properties is None:
            properties = self.implemented_properties

        Calculator.calculate(self, atoms, properties, system_changes)
        e_total, f_atoms = self.lj_core.calculate(atoms.get_atomic_numbers(), atoms.get_positions())
        self.results["energy"] = e_total
        self.results["free_energy"] = e_total
        self.results["forces"] = -f_atoms
        

def run_ase():
    calculator = LennardJonesI22Calculator()

    atomic_numbers = np.array([0, 1, 2, 3, 4, 5, 6, 7] * 8)
    positions = np.array([[3.5 * x, 0.0, 0.0] for x in range(len(atomic_numbers))])

    atoms = ase.Atoms(numbers=atomic_numbers, positions=positions)
    atoms.calc = calculator

    opt = ase.optimize.BFGS(atoms, logfile=None)
    opt.run(fmax=0.001)
    
    print("output: {:.5f}".format(atoms.get_potential_energy()))

    
run_ase()

output: -26.69947


## Supplementary: Detailed Function Forms

The following information is not necessary to solve the problem, but is provided for those who want to know what is inside. If there is a mistake in the following description, please assume the above implementation is correct.

The Lennard-Jones potential is two-body potential expressed in the following form when the interatomic distance is r.

$$
U(r)=4\epsilon \left\{\left(\frac{\sigma}{r}\right)^{12}-\left(\frac{\sigma}{r}\right)^{6}\right\}
$$

ε and σ are constants. The first term in parentheses is the repulsive force proportional to the -12th power of the distance and the second term is the attractive force proportional to the -6th power of the distance. ε is a constant for the strength of the bond and σ is a constant for the distance scale.

The behavior of the function is that it diverges to infinity when r asymptotes to zero, and it asymptotes to zero at the infinity of r. Differentiation shows that it reaches a minimum approximately where r is 1.12 times σ.

The values of ε and σ are determined by the type of atoms (=labels of points) on both sides. In this case, we have given them explicitly as follows.

```
sigma_single = np.array([2.0 + 0.2 * x for x in range(8)])
epsilon_single = np.array([0.1 + 0.1 * x for x in range(8)])

self.sigma_matrix = 0.5 * (sigma_single[None, :] + sigma_single[:, None])
self.epsilon_matrix = np.sqrt(epsilon_single[None, :] * epsilon_single[:, None])

```

The top two lines determine ε and σ between same atom types. The ε and σ between different atom types are synergistically averaged for ε and additively averaged for σ. This way of determining the parameters for different types of atoms is called the Lorentz-Berthelot rule in the Lennard-Jones potential.